# Variables Globales 

In [ ]:
import pysolr
import pandas as pd
import requests
import re

# Descargamos el Corpus 

In [48]:
files_music = [{'id_file':'1UuvbgIehXY-Rh1n80WeuAQ-WS0lRam6E', 'type':'vallenato'},
               {'id_file':'1V4NHXqlg2RDhZzlg87NLCvrra2csfqA1', 'type':'rock'},
               {'id_file':'1UsIOKTUfX-JeTF-82I1XgHvPdHgm5nki', 'type':'pop'},
               {'id_file':'1VCNdqcelornf6P8YmyNW1BcdoU30qfgO', 'type':'popular'},
               {'id_file':'1VCBzL6qySO4-a3tS0jwqqFQ3nHZuh46Q', 'type':'regueton'}]
df_music = {}
for item in files_music:
  # URL del archivo JSON en Google Drive
  file_id = item['id_file']
  url = f'https://drive.google.com/uc?export=download&id={file_id}'

  # Descargar el archivo JSON
  response = requests.get(url)
  response.raise_for_status()  # Check for request errors

  # Leer el contenido del archivo JSON en un DataFrame de Pandas
  data = response.json()  # Parse JSON from the response
  df = pd.json_normalize(data)
  df['type'] = item['type']

  # Mostrar el DataFrame
  print(item['type'], df.shape)
  df_music[item['type']] = df

vallenato (800, 7)
rock (503, 7)
pop (1829, 7)
popular (189, 7)
regueton (1638, 7)


# Métodos para Interactuar con Solr

In [49]:
def add_documents_to_solr(core_url, documents):
    """
    Inserta documentos en el core de Solr.

    Parameters:
    core_url (str): La URL completa del core de Solr, por ejemplo, 'http://localhost:8983/solr/lyricrs_example'.
    documents (list): Lista de documentos a insertar en el formato [{campo: valor}, {campo: valor}, ...].
    """
    try:
        # Conectarse al core
        solr = pysolr.Solr(core_url, always_commit=True, timeout=10)

        # Añadir documentos
        solr.add(documents)

        print("Documentos agregados exitosamente.")

    except Exception as e:
        print(f"Error al agregar documentos: {e}")



In [53]:
def add_dataframe_to_solr(core_url, dataframe):
    """
    Convierte un DataFrame de pandas en documentos JSON y los ingresa en Solr.

    Parameters:
    core_url (str): La URL completa del core de Solr, por ejemplo, 'http://localhost:8983/solr/lyricrs_example'.
    dataframe (pd.DataFrame): DataFrame de pandas que contiene los datos a insertar.
    """
    try:
        # Conectarse al core
        solr = pysolr.Solr(core_url, always_commit=True, timeout=10)

        # Convertir el DataFrame a una lista de diccionarios (JSON-like)
        documents = dataframe.to_dict(orient='records')
        
        # Añadir documentos a Solr
        solr.add(documents)

        print(f"{len(documents)} Documentos agregados exitosamente.")

    except Exception as e:
        print(f"Error al agregar documentos: {e}")

In [57]:
def search_song_lyrics(core_url, search_term):
    """
    Consulta las canciones en Solr por el contenido de las letras (lyrics) y
    devuelve la canción que contiene el término de búsqueda.
    """
    try:
        # Conectarse al core de Solr
        solr = pysolr.Solr(core_url, always_commit=True, timeout=10)

        # Definir los parámetros de la consulta
        params = {
            'hl': 'true',  # Activar el resaltado de texto
            'hl.fl': 'lyrics',  # Resaltar coincidencias en el campo 'lyrics'
            'hl.snippets': 2,  # Mostrar dos frases donde se encontró el término
            'hl.simple.pre': '<b>',
            'hl.simple.post': '</b>',
            'fl': '* score'  # Incluir todos los campos y el score
        }

        # Realizar la búsqueda en el campo "lyrics"
        results = solr.search(f"lyrics:{search_term}", **params)

        # Procesar los resultados
        processed_results = []

        for result in results:
            # Verificar si existe el resaltado para el documento
            highlighted_lyrics = results.highlighting.get(result['id'], {}).get('lyrics', [])

            # Limpiar las etiquetas <b> y </b> del texto resaltado
            clean_lyrics = [re.sub(r'<.*?>', '', frag)[:200] + '...' for frag in
                            highlighted_lyrics]  # Limitar a 200 caracteres

            song_data = {
                'id': result.get('id'),
                'title': result.get('title')[0] if result.get('title') else 'N/A',
                'artists': result.get('artists')[0] if result.get('artists') else 'N/A',
                'language': result.get('language')[0] if result.get('language') else 'N/A',
                'url': result.get('url')[0] if result.get('url') else 'N/A',
                'type': result.get('type')[0] if result.get('type') else 'N/A',
                'lyrics_highlighted': clean_lyrics if clean_lyrics else ["No hay fragmentos destacados"],
                'score': result.get('score')
            }
            processed_results.append(song_data)

        return processed_results

    except Exception as e:
        print(f"Error al realizar la consulta: {e}")
        return []


In [59]:
def delete_all_documents(core_url):
    solr = pysolr.Solr(core_url, always_commit=True, timeout=10)
    solr.delete(q='*:*')
    print("Todos los documentos han sido eliminados.")



Todos los documentos han sido eliminados.


# Agregamos los datos a Solr

In [50]:
df_music['regueton']

,url,title,artists,language,lyrics,_id.$oid,type
0,https://www.musica.com/letras.asp?letra=2533171,Mood Remix,"24kGoldn,J Balvin,Justin Bieber",en,"Woah\nJB, JB Why you always in a mood? Fuckin'...",64d50bf988bd5d509413d40b,regueton
1,https://www.musica.com/letras.asp?letra=2533172,Mood Remix,"24kGoldn,J Balvin,Justin Bieber",en,"Woah\nJB, JB Why you always in a mood? Fuckin'...",64d50bf988bd5d509413d40c,regueton
2,https://www.musica.com/letras.asp?letra=2533173,Mood Remix,"24kGoldn,J Balvin,Justin Bieber",en,"Woah\nJB, JB Why you always in a mood? Fuckin'...",64d50bfa88bd5d509413d40d,regueton
3,https://www.musica.com/letras.asp?letra=2439752,Bola Rebola,"Anitta,J Balvin,Tropkillaz,MC Zaac",es,"[MC Zaac]\nÊ, mulher, ah\nVai assim, tá bom?\n...",64d4a60c88bd5d5094129254,regueton
4,https://www.musica.com/letras.asp?letra=2440169,Bola Rebola,"Anitta,J Balvin,Tropkillaz,MC Zaac",es,"[MC Zaac]\nÊ, mulher, ah\nVai assim, tá bom?\n...",64d4a67788bd5d50941293d2,regueton
...,...,...,...,...,...,...,...
1633,https://www.musica.com/letras.asp?letra=2585071,Que Mas Pues,"Ryan Castro,Sir Speedy",es,"Oh-oh\nMamacita\nCoge la curva, mami Y fue por...",64d55a7688bd5d5094145b77,regueton
1634,https://www.musica.com/letras.asp?letra=2585079,Que Mas Pues,"Ryan Castro,Sir Speedy",es,"Oh-oh\nMamacita\nCoge la curva, mami Y fue por...",64d55a7888bd5d5094145b7f,regueton
1635,https://www.musica.com/letras.asp?letra=2570725,Monumento Remix,"Ryan Castro,Ñejo,Andy Rivera",es,Quédate cinco minutos más\nUn polvo más nunca ...,64d53f9388bd5d50941435f4,regueton
1636,https://www.musica.com/letras.asp?letra=2570731,Monumento Remix,"Ryan Castro,Ñejo,Andy Rivera",es,Quédate cinco minutos más\nUn polvo más nunca ...,64d53f9588bd5d50941435fa,regueton


In [51]:
df_music['regueton'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1638 entries, 0 to 1637
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   url       1638 non-null   object
 1   title     1638 non-null   object
 2   artists   1638 non-null   object
 3   language  1638 non-null   object
 4   lyrics    1638 non-null   object
 5   _id.$oid  1638 non-null   object
 6   type      1638 non-null   object
dtypes: object(7)
memory usage: 89.7+ KB


In [52]:
core_url = 'http://localhost:8983/solr/lyrics_example'

In [60]:
add_dataframe_to_solr(core_url, df_music['regueton'])

1638 Documentos agregados exitosamente.


# Realizamos operaciones especificas

In [58]:
url_cancion = "https://www.letras.com/j-balvin/abrazame/"
    
search_term = """Nuevamente te busqué Y de ti no encontré nada"""  # Cambia esto por el término de búsqueda que desees

# Realizar la búsqueda
results = search_song_lyrics(core_url, search_term)

print(results)


[{'id': 'a2379229-9381-4216-b015-fb9ceaa0817b', 'title': 'Abrázame ', 'artists': 'J Balvin', 'language': 'es', 'url': 'https://www.musica.com/letras.asp?letra=2083739', 'type': 'regueton', 'lyrics_highlighted': ['Nuevamente te extrañé\nNuevamente te busqué\nY de ti no encontré nada\nYa no escucho ni tu voz\nYa no aguanto este dolor\nHay que tengo me mata Abrázame aunque no estés a mi lado\nNiña siénteme\nMe tienes de...'], 'score': 4.014496}, {'id': 'e3dcac6d-2333-49e0-b14f-ce657c0ec0c6', 'title': 'Abrázame ', 'artists': 'J Balvin', 'language': 'es', 'url': 'https://www.musica.com/letras.asp?letra=2083739', 'type': 'regueton', 'lyrics_highlighted': ['Nuevamente te extrañé\nNuevamente te busqué\nY de ti no encontré nada\nYa no escucho ni tu voz\nYa no aguanto este dolor\nHay que tengo me mata Abrázame aunque no estés a mi lado\nNiña siénteme\nMe tienes de...'], 'score': 4.014496}, {'id': '85ff7641-98a2-46d6-910f-e4004ec82f73', 'title': 'No me vuelvo a enamorar ', 'artists': 'J Balvin', 

In [ ]:
delete_all_documents(core_url)